## Imports

In [1]:
import pandas as pd

from functools import partial
from bokeh.plotting import figure, show, output_notebook, ColumnDataSource
from bokeh.tile_providers import get_provider, Vendors
from pyproj import CRS, Transformer, Proj, transform
from shapely import ops
from shapely.geometry import Point, LineString, Polygon

## Set up data frames

In [2]:
pt_columns = ['Name', 'lat', 'lon']
pt_data = [["Left", 37+50/60, -92-50/60],
           ["Right", 42+22/60, -74-42/60]]
pt_df = pd.DataFrame(pt_data, columns=pt_columns)

In [3]:
line_columns = ['Name', 'init_lat', 'init_lon', 'final_lat', 'final_lon']
line_data = [['L to R', pt_data[0][1], pt_data[0][2], pt_data[1][1], pt_data[1][2]],
            ['CircleB', 35, -64.5, 36, -84.5],
            ['Cross', 20, -75, 38, -70],
            ['Miss', 40, -65, 40, -85]]
line_df = pd.DataFrame(line_data, columns=line_columns)
line_df['lat'] = line_df.apply(lambda row: [row.init_lat, row.final_lat], axis=1)
line_df['lon'] = line_df.apply(lambda row: [row.init_lon, row.final_lon], axis=1)

In [4]:
poly_df = pd.DataFrame(pt_data, columns=pt_columns)

## Map Configs

In [5]:
tile = get_provider(Vendors.STAMEN_TERRAIN)
output_notebook()

crs = CRS("world_mercator")

proj_wgs84 = Transformer.from_crs(crs.geodetic_crs, crs, always_xy=True)
m = 'mercator'

Loading BokehJS ...

## Transformations

## Geodetic to Mercator XY

In [6]:
def transform_to_xy(row):
    return proj_wgs84.transform(row.lon, row.lat)

def insert_xy(df):
    df['x'], df['y'] = zip(*df.apply(transform_to_xy, axis=1))

## Geodetic to Point Buffer

In [7]:
def geodesic_point_buffer(lat, lon, buffer_nm=0, buffer_km=0):
    # Azimuthal equidistant projection
    meters = buffer_nm * 1852 + buffer_km * 1000
    aeqd_proj = f'+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
    aeqd_to_wgs84 = partial(
        transform,
        Proj(aeqd_proj),
        crs.geodetic_crs)
    
    buffer = Point(0, 0).buffer(meters)    
    return ops.transform(aeqd_to_wgs84, buffer).exterior.coords[:]

def transform_point(df, buffer_nm=0, buffer_km=0):
    df['lines'] = df.apply(lambda row: geodesic_point_buffer(row.lat, row.lon, buffer_nm=buffer_nm, buffer_km=buffer_km),
                           axis=1)
    df['lat'] = df['lines'].apply(lambda x: [lonlat[1] for lonlat in x])
    df['lon'] = df['lines'].apply(lambda x: [lonlat[0] for lonlat in x])

## Geodetic to Line

In [8]:
def geodesic_line(start_point, end_point, tolerance=1000):
    start_x, start_y = start_point
    end_x, end_y = end_point
    
    g = crs.geodetic_crs.get_geod()
    
    az12, az21, dist = g.inv(start_x, start_y, end_x, end_y)
    
    return [start_point] + g.npts(start_x, start_y, end_x, end_y, 1 + int(dist / tolerance)) + [end_point]

def transform_line(df):
    df['geoline'] = df.apply(lambda row: geodesic_line((row.init_lon, row.init_lat), (row.final_lon, row.final_lat)), axis=1)
    df['lat'] = line_df['geoline'].apply(lambda x: [lonlat[1] for lonlat in x])
    df['lon'] = line_df['geoline'].apply(lambda x: [lonlat[0] for lonlat in x])

## Derive lines and polygons

In [9]:
transform_point(poly_df, buffer_nm=250)
transform_line(line_df)

## Add x,y columns to data frames for ColumnDataSource

In [10]:
insert_xy(pt_df)
insert_xy(line_df)
insert_xy(poly_df)

## Make a rectangle for both circles using xy's

In [11]:
def convex_hull(df):
    xs = [val for row in df.x.values for val in row]
    ys = [val for row in df.y.values for val in row]
    poly = Polygon([(x, y) for x,y in zip(xs,ys)])
    coords = poly.convex_hull.exterior.coords[:]
    xs = [xy[0] for xy in coords]
    ys = [xy[1] for xy in coords]
    
    return pd.DataFrame([['Hull', xs, ys]], columns=['Name', 'x', 'y'])    
    
def make_bounding_box(df):
    ys = []
    xs = []
    
    for i in range(len(poly_df.y)):
        ys.extend([min(poly_df.y[i]), max(poly_df.y[i])])
        imin = poly_df.y[i].index(ys[-2])
        imax = poly_df.y[i].index(ys[-1])
        xs.extend([poly_df.x[i][imin], poly_df.x[i][imax]])
        
    ys.append(ys.pop(1))
    xs.append(xs.pop(1))
    
    return pd.DataFrame([['Box1', xs, ys]], columns=['Name', 'x', 'y'])
    
newbox_df = convex_hull(poly_df)

## Make some geometries

In [12]:
pt_df['geometry'] = pt_df.apply(lambda row: Point(row.x, row.y), axis=1)
line_df['geometry'] = line_df.apply(lambda row: LineString([(a[0], a[1]) for a in zip(row.x, row.y)]), axis=1)
poly_df['geometry'] = poly_df.apply(lambda row: Polygon([(a[0], a[1]) for a in zip(row.x, row.y)]), axis=1)
newbox_df['geometry'] = newbox_df.apply(lambda row: Polygon([(a[0], a[1]) for a in zip(row.x, row.y)]), axis=1)

In [13]:
def cross_any(line):
    box_cross = [geom.crosses(line) for geom in newbox_df.geometry]
    circle_cross = [geom.crosses(line) for geom in poly_df.geometry]
    return True in box_cross or True in circle_cross    

line_df['crossed'] = line_df.geometry.apply(lambda line: cross_any(line))

## Setup column data source -- drop geometry columns (not serializable)

In [14]:
pt_ds = ColumnDataSource(pt_df.drop('geometry', axis=1))
line_out_ds = ColumnDataSource(line_df[~(line_df.crossed)].drop('geometry', axis=1))
line_in_ds = ColumnDataSource(line_df[line_df.crossed].drop('geometry', axis=1))
poly_ds = ColumnDataSource(poly_df.drop('geometry', axis=1))
newbox_ds = ColumnDataSource(newbox_df.drop('geometry', axis=1))

## Display!

In [15]:
offset=1000000

x_range = [pt_df['x'].min()-offset, pt_df['x'].max()+offset]
y_range = [pt_df['y'].min(), pt_df['y'].max()]

p = figure(x_range=x_range,
           y_range=y_range, 
           x_axis_type=m, 
           y_axis_type=m, 
           title="Test Plot - 2 Circles and a Box!",
           x_axis_label="Longitude",
           y_axis_label="Latitude",
           plot_width=800, 
           plot_height=800
          )
p.add_tile(tile)

p.circle('x', 'y', name="Points", radius=50000, source=pt_ds, alpha=0.5, fill_color="orange")
p.multi_line('x', 'y', name="Line", source=line_in_ds, alpha=1.0, line_width=2, line_color="indigo")
p.multi_line('x', 'y', name="Line", source=line_out_ds, alpha=0.25, line_color="indigo")
p.patches('x', 'y', name="Poly", source=poly_ds, alpha=0.25, fill_color='green')
p.patches('x', 'y', name="NewBox", source=newbox_ds, alpha=0.25, fill_color='yellow')

show(p)